In [4]:
import os
import librosa
import parselmouth
import numpy as np
import pandas as pd

# Function to extract pitch using librosa
def extract_pitch(file_path):
    try:
        y, sr = librosa.load(file_path, sr=None)
        pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
        pitches = pitches[np.nonzero(pitches)]  # Filter non-zero pitch values
        if len(pitches) > 0:
            return np.mean(pitches)
        else:
            return 0
    except Exception as e:
        print(f"Error extracting pitch for {file_path}: {e}")
        return None

# Function to extract jitter and shimmer using parselmouth
def extract_jitter_shimmer(file_path):
    try:
        sound = parselmouth.Sound(file_path)
        pitch = sound.to_pitch(pitch_floor=75, pitch_ceiling=500)  # Adjust pitch range

        # Generate PointProcess for voiced parts
        point_process = parselmouth.praat.call(sound, "To PointProcess (periodic, cc)", 75, 500)

        # Start and end times for jitter and shimmer computation
        start_time = 0.0
        end_time = sound.get_total_duration()  # Total audio duration
        shortest_period = 0.0001  # Minimum pitch period (100 Hz)
        longest_period = 0.02  # Maximum pitch period (50 Hz)

        # Extract jitter (local)
        jitter = parselmouth.praat.call(
            point_process, "Get jitter (local)", start_time, end_time, shortest_period, longest_period
        )

        # Extract shimmer (local)
        shimmer = parselmouth.praat.call(
            [sound, point_process], "Get shimmer (local)", start_time, end_time, shortest_period, longest_period
        )

        return jitter, shimmer
    except Exception as e:
        print(f"Error extracting jitter/shimmer for {file_path}: {e}")
        return None, None

# Directory structure
base_directory = "Audio_Speech_Actors_01-24"
output_data = []

# Traverse through the directories
for subdir in sorted(os.listdir(base_directory)):
    subdir_path = os.path.join(base_directory, subdir)
    if os.path.isdir(subdir_path):
        for file in os.listdir(subdir_path):
            if file.endswith(".wav"):
                file_path = os.path.join(subdir_path, file)
                try:
                    # Extract features
                    pitch = extract_pitch(file_path)
                    jitter, shimmer = extract_jitter_shimmer(file_path)
                    
                    # Append results to the data list
                    output_data.append({
                        "Actor": subdir,
                        "File": file,
                        "Pitch": pitch if pitch is not None else 0,
                        "Jitter": jitter if jitter is not None else 0,
                        "Shimmer": shimmer if shimmer is not None else 0
                    })
                    print(f"Processed {file_path}")
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")

# Convert the data to a DataFrame and save as CSV
output_df = pd.DataFrame(output_data)
output_df.to_csv("audio_features_2.csv", index=False)
print("Feature extraction complete. Results saved to audio_features.csv.")


Error extracting jitter/shimmer for Audio_Speech_Actors_01-24/Actor_01/03-01-08-02-02-01-01.wav: Command requires more than the given 4 arguments: no value for argument "Maximum period factor".
Processed Audio_Speech_Actors_01-24/Actor_01/03-01-08-02-02-01-01.wav
Error extracting jitter/shimmer for Audio_Speech_Actors_01-24/Actor_01/03-01-08-01-01-01-01.wav: Command requires more than the given 4 arguments: no value for argument "Maximum period factor".
Processed Audio_Speech_Actors_01-24/Actor_01/03-01-08-01-01-01-01.wav
Error extracting jitter/shimmer for Audio_Speech_Actors_01-24/Actor_01/03-01-05-01-02-01-01.wav: Command requires more than the given 4 arguments: no value for argument "Maximum period factor".
Processed Audio_Speech_Actors_01-24/Actor_01/03-01-05-01-02-01-01.wav
Error extracting jitter/shimmer for Audio_Speech_Actors_01-24/Actor_01/03-01-06-01-02-02-01.wav: Command requires more than the given 4 arguments: no value for argument "Maximum period factor".
Processed Audi

In [5]:
file_path = "Audio_Speech_Actors_01-24/Actor_01/03-01-08-02-02-01-01.wav"
pitch = extract_pitch(file_path)
jitter, shimmer = extract_jitter_shimmer(file_path)
print(f"Pitch: {pitch}, Jitter: {jitter}, Shimmer: {shimmer}")


Error extracting jitter/shimmer for Audio_Speech_Actors_01-24/Actor_01/03-01-08-02-02-01-01.wav: Command requires more than the given 4 arguments: no value for argument "Maximum period factor".
Pitch: 1899.7996826171875, Jitter: None, Shimmer: None


In [6]:
def extract_jitter_shimmer2(file_path):
    try:
        sound = parselmouth.Sound(file_path)
        print(f"Sound duration: {sound.get_total_duration()}")  # Debug: Ensure valid audio duration
        
        # Convert sound to pitch
        pitch = sound.to_pitch(pitch_floor=75, pitch_ceiling=500)
        if not pitch:
            raise ValueError("Failed to extract pitch: pitch object is None")
        
        # Generate PointProcess
        point_process = parselmouth.praat.call(sound, "To PointProcess (periodic, cc)", 75, 500)
        
        # Extract jitter and shimmer with proper arguments
        jitter = parselmouth.praat.call(
            point_process, "Get jitter (local)", 0.0, sound.get_total_duration(), 0.0001, 0.02, 1.3
        )
        shimmer = parselmouth.praat.call(
            [sound, point_process], "Get shimmer (local)", 0.0, sound.get_total_duration(), 0.0001, 0.02, 1.3
        )
        return jitter, shimmer
    except Exception as e:
        print(f"Error extracting jitter/shimmer for {file_path}: {e}")
        return None, None


file_path = "Audio_Speech_Actors_01-24/Actor_01/03-01-08-02-02-01-01.wav"
pitch = extract_pitch(file_path)
jitter, shimmer = extract_jitter_shimmer2(file_path)
print(f"Pitch: {pitch}, Jitter: {jitter}, Shimmer: {shimmer}")


Sound duration: 3.5035
Error extracting jitter/shimmer for Audio_Speech_Actors_01-24/Actor_01/03-01-08-02-02-01-01.wav: Command requires more than the given 5 arguments: no value for argument "Maximum amplitude factor".
Pitch: 1899.7996826171875, Jitter: None, Shimmer: None


In [2]:
def extract_jitter_shimmer3(file_path):
    try:
        sound = parselmouth.Sound(file_path)
        print(f"Sound duration: {sound.get_total_duration()}")  # Debug: Ensure valid audio duration

        # Convert sound to pitch
        pitch = sound.to_pitch(pitch_floor=75, pitch_ceiling=500)
        if not pitch or len(pitch.selected_array['frequency']) == 0:
            raise ValueError("Failed to extract pitch: No voiced segments detected")
        
        # Generate PointProcess
        point_process = parselmouth.praat.call(sound, "To PointProcess (periodic, cc)", 75, 500)

        # Extract jitter
        jitter = parselmouth.praat.call(
            point_process, "Get jitter (local)", 0.0, sound.get_total_duration(), 0.0001, 0.02, 1.3
        )

        # Extract shimmer
        shimmer = parselmouth.praat.call(
            [sound, point_process], "Get shimmer (local)", 0.0, sound.get_total_duration(), 0.0001, 0.02, 1.3, 1.6
        )

        return jitter, shimmer
    except Exception as e:
        print(f"Error extracting jitter/shimmer for {file_path}: {e}")
        return None, None
    
def extract_pitch(file_path):
    try:
        y, sr = librosa.load(file_path, sr=None)
        pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
        pitches = pitches[np.nonzero(pitches)]  # Filter non-zero pitch values
        if len(pitches) > 0:
            return np.mean(pitches)
        else:
            return 0
    except Exception as e:
        print(f"Error extracting pitch for {file_path}: {e}")
        return None
    

file_path = "Audio_Speech_Actors_01-24/Actor_01/03-01-08-02-02-01-01.wav"
pitch = extract_pitch(file_path)
jitter, shimmer = extract_jitter_shimmer3(file_path)
print(f"Pitch: {pitch}, Jitter: {jitter}, Shimmer: {shimmer}")


Error extracting pitch for Audio_Speech_Actors_01-24/Actor_01/03-01-08-02-02-01-01.wav: name 'librosa' is not defined
Error extracting jitter/shimmer for Audio_Speech_Actors_01-24/Actor_01/03-01-08-02-02-01-01.wav: name 'parselmouth' is not defined
Pitch: None, Jitter: None, Shimmer: None
